In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        #5x5 kernal on 28x28 image. Should have 2 padding for "32x32" image
        self.c1 = nn.Conv2d(in_channels=1, kernel_size=5, padding=2, out_channels=6) #results in 28x28 in 6 channels. Should have 1 channel-in bc it is one image at first
        #Relu 28x28 -> 28x28
        #Max pool 28x28 -> 14x14 (stride=2)
        self.c2 = nn.Conv2d(in_channels=6, kernel_size=5, out_channels=16) #6 channels to 16 channels. 14x14 -> 10x10 with 5x5kernel
        #relu 10x10->10x10
        #max pool 10x10 -> 5x5
        self.fc1 = nn.Linear(25*16, 120) #5x5 images, 16 channels in. 120 out
        self.fc2 = nn.Linear(120, 84) #120 -> 84
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #C1 Relu max pool
        x = F.relu(self.c1(x))
        x = F.max_pool2d(x, kernel_size=(2,2), stride=(2,2))
        #c2 relu max pool
        x = F.relu(self.c2(x))
        x = F.max_pool2d(x, kernel_size=(2,2), stride=(2,2))
        #fc layers
        x = x.view(-1, 5*5*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def name(self):
        return "LeNet"